In [1]:
pip install matplotlib

DEPRECATION: celery 4.4.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [2]:
# brew install portaudio


In [3]:
pip install pyaudio

DEPRECATION: celery 4.4.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install speech_recognition

ERROR: Could not find a version that satisfies the requirement speech_recognition (from versions: none)
ERROR: No matching distribution found for speech_recognition
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install python_speech_features

DEPRECATION: celery 4.4.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install noisereduce

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/a9/cc/5ed2280a27e5dab12994c884f1f4d8c3bd4d885d02ae9e52a9d213a6a5e2/numpy-1.24.4-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Using cached numpy-1.24.4-cp311-cp311-macosx_10_9_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-macosx_10_9_x86_64.whl (19.8 MB)
DEPRECATION: celery 4.4.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.0
    Uninstalling numpy-1.26.0:
      Successfully uninstalled numpy-1.26.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [7]:
pip install numpy==1.24

  Using cached numpy-1.24.0-cp311-cp311-macosx_10_9_x86_64.whl (19.8 MB)
DEPRECATION: celery 4.4.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
seaborn 0.12.2 requires numpy!=1.24.0,>=1.17, but you have numpy 1.24.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tensorflow

DEPRECATION: celery 4.4.5 has a non-standard dependency specifier pytz>dev. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [78]:
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal
import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')
import soundfile
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

C:\Users\HP\AppData\Local\Temp\ipykernel_10944\2007753442.py:16: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('matplotlib inline')


In [79]:
ravdess = 'Dataset'
ravdess_directory_list = os.listdir(ravdess)
ravdess_directory_list.remove('audio_speech_actors_01-24')
print(ravdess_directory_list)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [80]:
os.listdir(path='DataSet')

def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = 'Dataset'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

2049

In [81]:

#DOWN SAMPLING OF AUDIO FILES IS DONE  AND PUT MASK OVER IT AND DIRECT INTO CLEAN FOLDER
#MASK IS TO REMOVE UNNECESSARY EMPTY VOIVES AROUND THE MAIN AUDIO VOICE 
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [82]:
#DOWN SAMPLING OF AUDIO FILES IS DONE  AND PUT MASK OVER IT AND DIRECT INTO CLEAN FOLDER
#MASK IS TO REMOVE UNNECESSARY EMPTY VOIVES AROUND THE MAIN AUDIO VOICE 
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [83]:
#The clean Audio Files are redirected to Clean Audio Folder Directory 
import glob,pickle
for file in tqdm(glob.glob(r'DataSet/**/*.wav')):
    file_name = os.path.basename(file)
    signal , rate = librosa.load(file, sr=16000)
    mask = envelope(signal,rate, 0.0005)
    wavfile.write(filename= r'CleanData/'+str(file_name), rate=rate,data=signal[mask])


  0%|          | 0/1439 [00:00<?, ?it/s]

100%|██████████| 1439/1439 [00:40<00:00, 35.17it/s]


In [85]:
signal

array([0.        , 0.        , 0.        , ..., 0.00012292, 0.00013861,
       0.        ], dtype=float32)

In [86]:

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        result=np.hstack((result, mel))
        # print('yes')
    return result


In [87]:

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}


In [88]:
#Load the data and extract features for each sound file
from glob import glob
import os
import glob
def load_data(test_size=0.33):
    x,y=[],[]
    answer = 0
    for file in glob.glob(r'CleanData/*.wav'):
        file_name=os.path.basename(file)
        # print(file)
        emotion=emotions[file_name.split("-")[2]]
        # print(emotion)
        # if emotion not in emotions:
        #     answer += 1
        #     # print(answer)
        #     continue
        # print(answer)
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        # print(feature)
        x.append(feature)
        y.append([emotion,file_name])
        # print(x,1)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=0)

In [89]:
#Split the dataset
import librosa
import numpy as np
x_train,x_test,y_trai,y_tes=load_data(test_size=0.25)
print(np.shape(x_train),np.shape(x_test), np.shape(y_trai),np.shape(y_tes))
y_test_map = np.array(y_tes).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_trai).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train),np.shape(y_test))
print(*test_filename,sep="\n")

(1080, 180) (360, 180) (1080, 2) (360, 2)
(1080,) (360,)
03-01-01-01-01-02-11.wav
03-01-07-01-01-01-06.wav
03-01-08-01-01-01-09.wav
03-01-03-01-01-02-22.wav
03-01-02-01-01-02-22.wav
03-01-03-01-02-02-05.wav
03-01-04-01-02-01-21.wav
03-01-02-02-02-02-15.wav
03-01-03-02-02-02-12.wav
03-01-07-01-02-01-24.wav
03-01-05-01-01-01-08.wav
03-01-04-02-02-01-19.wav
03-01-02-01-02-01-09.wav
03-01-05-01-01-01-15.wav
03-01-08-02-01-02-13.wav
03-01-02-02-02-02-05.wav
03-01-05-01-01-02-21.wav
03-01-06-02-01-02-01.wav
03-01-06-02-02-01-07.wav
03-01-08-01-02-01-02.wav
03-01-02-01-02-02-15.wav
03-01-02-02-02-01-15.wav
03-01-04-02-02-02-04.wav
03-01-08-02-02-02-06.wav
03-01-08-02-02-01-18.wav
03-01-05-02-02-02-14.wav
03-01-02-01-01-02-23.wav
03-01-02-02-01-02-09.wav
03-01-06-02-02-02-23.wav
03-01-02-02-01-02-08.wav
03-01-01-01-01-02-08.wav
03-01-02-02-01-01-11.wav
03-01-06-02-02-01-19.wav
03-01-03-01-01-01-12.wav
03-01-07-01-01-01-03.wav
03-01-05-01-01-02-18.wav
03-01-07-01-02-01-13.wav
03-01-05-02-02-01-

In [14]:
#Get the shape of the training and testing datasets
# print((x_train.shape[0], x_test.shape[0]))
print((x_train[0], x_test[0]))
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

(array([-2.79258057e+02,  4.44856949e+01, -2.75269146e+01,  3.96854568e+00,
       -8.96938229e+00, -5.84686947e+00, -3.13966599e+01, -1.15887756e+01,
       -2.22256260e+01,  1.00622988e+00, -1.50275049e+01, -6.48051834e+00,
       -1.11798775e+00, -1.89276171e+00,  6.62633002e-01, -5.37966585e+00,
       -8.74827802e-01,  4.55192375e+00, -5.80102921e+00,  5.07920742e+00,
        2.68067384e+00,  3.73211527e+00,  1.03725624e+01,  1.42534218e+01,
        8.92649651e+00,  1.26670399e+01,  6.47839308e+00,  1.01897135e+01,
        3.76886106e+00,  5.03343534e+00,  1.88534713e+00,  3.67499757e+00,
        4.16484404e+00,  3.79482222e+00, -2.00221205e+00, -3.68215501e-01,
       -3.26937604e+00,  1.58522713e+00,  1.21718776e+00, -8.58303428e-01,
        5.34178913e-01,  6.27427340e-01,  5.89389682e-01,  5.67165256e-01,
        5.93753159e-01,  6.27108693e-01,  6.22477591e-01,  6.42424405e-01,
        6.35390103e-01,  6.33210301e-01,  5.77480853e-01,  4.92184401e-01,
        3.79147500e-01, 

In [90]:
from sklearn.preprocessing import StandardScaler

In [91]:
# scaling our data with sklearn's Standard scaler
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1080, 180), (1080,), (360, 180), (360,))

In [92]:

# Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [93]:
# Define your mapping dictionary
emotion_dict = {'neutral': 0, 'calm': 1, 'happy': 2, 'sad': 3, 'angry': 4, 'fearful': 5, 'disgust': 6, 'surprised': 7}

# Vectorize the function so it can be applied element-wise to a numpy array
vfunc = np.vectorize(lambda x: emotion_dict[x])

# Apply the function to your array
y_train_encoded = vfunc(y_train)
y_test_encoded = vfunc(y_test)

In [94]:
#Train the model
model.fit(x_train,y_train_encoded)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [95]:
model.score(x_test,y_test_encoded)

0.65

In [96]:
#SAVING THE MODEL
import pickle
# Save the Modle to file in the current working directory
#For any new testing data other than the data in dataset

Pkl_Filename = "Emotion_Voice_Detection_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [97]:
with open(Pkl_Filename, 'rb') as file:  
    Emotion_Voice_Detection_Model = pickle.load(file)

Emotion_Voice_Detection_Model

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [98]:
y_pred=Emotion_Voice_Detection_Model.predict(x_test)
y_pred

array([3, 1, 7, 2, 1, 6, 6, 1, 2, 2, 2, 5, 1, 4, 7, 1, 5, 5, 5, 7, 1, 1,
       3, 3, 7, 4, 1, 1, 2, 1, 0, 1, 5, 2, 6, 4, 6, 4, 1, 2, 1, 3, 1, 1,
       5, 6, 1, 7, 4, 3, 6, 1, 4, 1, 1, 3, 0, 5, 4, 7, 1, 7, 7, 3, 3, 3,
       4, 0, 2, 1, 2, 5, 6, 7, 3, 4, 3, 2, 7, 6, 6, 2, 2, 7, 4, 5, 6, 0,
       6, 6, 5, 7, 5, 2, 6, 7, 1, 2, 7, 5, 2, 6, 7, 1, 4, 5, 7, 6, 0, 1,
       6, 1, 5, 3, 5, 6, 2, 5, 1, 4, 6, 1, 6, 6, 3, 0, 7, 6, 0, 3, 6, 7,
       7, 0, 5, 1, 1, 5, 6, 5, 6, 7, 1, 4, 1, 6, 1, 3, 7, 5, 4, 7, 6, 4,
       7, 2, 0, 2, 5, 3, 4, 1, 5, 1, 0, 5, 2, 6, 6, 7, 6, 0, 4, 6, 6, 3,
       2, 7, 7, 0, 2, 6, 6, 3, 7, 3, 5, 3, 5, 5, 7, 4, 4, 5, 5, 2, 1, 4,
       3, 4, 2, 6, 2, 3, 1, 5, 0, 3, 6, 5, 1, 2, 3, 3, 3, 7, 6, 2, 6, 6,
       6, 3, 1, 4, 7, 2, 2, 3, 1, 6, 6, 5, 4, 0, 7, 6, 7, 3, 6, 5, 7, 3,
       2, 5, 5, 2, 1, 1, 7, 1, 4, 5, 5, 1, 4, 7, 6, 5, 3, 7, 2, 2, 3, 6,
       6, 0, 4, 2, 0, 4, 2, 3, 0, 2, 0, 2, 6, 3, 1, 7, 2, 4, 1, 1, 3, 5,
       7, 4, 3, 1, 5, 3, 6, 5, 5, 2, 7, 6, 7, 2, 4,

In [116]:
#RECORDED USING MICROPHONE:
import pyaudio
import wave

CHUNK = 1024 
FORMAT = pyaudio.paInt16 #paInt8
CHANNELS = 1
RATE = 44100 #sample rate
RECORD_SECONDS = 4
WAVE_OUTPUT_FILENAME = "output10"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK) #buffer

print("* recording")
frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data) # 2 bytes(16 bits) per channel

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [121]:
#The file 'output10.wav' in the next cell is the file that was recorded live using the code :
data, sampling_rate = librosa.load('DataSet//Actor_01//03-01-04-02-02-02-01.wav')
# %matplotlib inline
# import os
# import pandas as pd
# import librosa.display
# import glob 

# plt.figure(figsize=(15, 5))
# librosa.display.waveshow(data, sr=sampling_rate)

In [127]:
file = 'DataSet//Actor_01//03-01-08-02-02-02-01.wav'
# data , sr = librosa.load(file)
# data = np.array(data)
ans =[]
new_feature  = extract_feature(file, mfcc=True, chroma=True, mel=True)
ans.append(new_feature)
ans = np.array(ans)
ans_tr = scaler.transform(ans)
ans_tra = np.array(ans_tr)
# data.shape
Emotion_Voice_Detection_Model.predict([ans_tra][0])

array([5])

In [128]:
a = Emotion_Voice_Detection_Model.predict([ans_tra][0])


In [129]:
pred = str(a[0])

In [ ]:
pred

'5'

Exception ignored in: <function Wave_write.__del__ at 0x0000029C7C0AACA0>
Traceback (most recent call last):
  File "c:\Program Files\Python311\Lib\wave.py", line 447, in __del__
    self.close()
  File "c:\Program Files\Python311\Lib\wave.py", line 564, in close
    if self._file:
       ^^^^^^^^^^
AttributeError: 'Wave_write' object has no attribute '_file'
Exception ignored in: <function Wave_write.__del__ at 0x0000029C7C0AACA0>
Traceback (most recent call last):
  File "c:\Program Files\Python311\Lib\wave.py", line 447, in __del__
    self.close()
  File "c:\Program Files\Python311\Lib\wave.py", line 564, in close
    if self._file:
       ^^^^^^^^^^
AttributeError: 'Wave_write' object has no attribute '_file'
Traceback (most recent call last):
  File "C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\gradio\routes.py", line 516, in predict
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\HP\AppData\Roaming\

In [105]:
emotions={
  '0':'neutral',
  '1':'calm',
  '2':'happy',
  '3':'sad',
  '4':'angry',
  '5':'fearful',
  '6':'disgust',
  '7':'surprised'
}

In [106]:
emotions[pred]

'disgust'

In [107]:
import gradio as gr

In [108]:
def Predict(audio):
    ans  = []
    new_feature = extract_feature(audio, mfcc=True, chroma=True, mel=True)
    ans.append(new_feature)
    ans = np.array(ans)
    ans_tr = scaler.transform(ans)
    ans_tra = np.array(ans_tr)  
    a = Emotion_Voice_Detection_Model.predict([ans_tra][0])
    pred = str(a[0])
    return emotions[pred]

In [109]:
iface = gr.Interface(
    fn = Predict,
    inputs=gr.Audio(source="upload", type="filepath"),
    outputs = "text",
    live = True
)

In [110]:
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7863

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
